In [0]:
# some libraries
import pandas as pd
import numpy as np

In [0]:
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

In [0]:
from google.colab import files
uploaded = files.upload()

Saving aus_series.csv to aus_series.csv


In [0]:
# to read the dataframe
import io
df = pd.read_csv(io.StringIO(uploaded['aus_series.csv'].decode('utf-8')))
df.head()

,Date,AUS_TM_TAX_MANF_BR_ZS,AUS_TM_TAX_TCOM_BR_ZS,AUS_FM_LBL_BMNY_GD_ZS,AUS_FM_LBL_BMNY_CN,AUS_FM_LBL_BMNY_ZG,AUS_IQ_WEF_CUST_XQ,AUS_FM_LBL_BMNY_IR_ZS,AUS_IC_BUS_DISC_XQ,AUS_GC_BAL_CASH_GD_ZS,AUS_GC_BAL_CASH_CN,AUS_GC_DOD_TOTL_GD_ZS,AUS_GC_DOD_TOTL_CN,AUS_AG_PRD_CREL_MT,AUS_AG_YLD_CREL_KG,AUS_NE_GDI_STKB_KN,AUS_NE_GDI_STKB_CN,AUS_BN_RES_INCL_CD,AUS_NE_GDI_STKB_CD,AUS_NV_MNF_CHEM_ZS_UN,AUS_SH_HIV_0014,AUS_SE_PRM_UNER_MA,AUS_SE_PRM_UNER_FE,AUS_SE_PRM_UNER,AUS_FM_AST_CGOV_ZG_M3,AUS_FS_AST_CGOV_GD_ZS,AUS_FM_AST_DOMO_ZG_M3,AUS_FS_AST_DOMO_GD_ZS,AUS_FM_AST_PRVT_ZG_M3,AUS_EN_ATM_CO2E_PP_GD,AUS_EN_ATM_CO2E_KD_GD,AUS_EN_ATM_CO2E_PP_GD_KD,AUS_EN_ATM_CO2E_KT,AUS_EN_ATM_CO2E_PC,AUS_EN_CO2_ETOT_ZS,AUS_EN_CO2_ETOT_MT,AUS_EN_ATM_CO2E_GF_ZS,AUS_EN_ATM_CO2E_GF_KT,AUS_EN_ATM_CO2E_LF_KT,AUS_EN_ATM_CO2E_LF_ZS,...,AUS_TX_VAL_AGRI_ZS_UN,AUS_EN_ATM_NOXE_AG_KT_CE,AUS_TM_VAL_AGRI_ZS_UN,AUS_EA_PRD_AGRI_KD,AUS_NV_AGR_TOTL_ZS,AUS_NV_AGR_TOTL_KD_ZG,AUS_NV_AGR_TOTL_CN,AUS_NV_AGR_TOTL_KD,AUS_NV_AGR_TOTL_KN,AUS_IS_AIR_GOOD_MT_K1,AUS_NV_AGR_TOTL_CD,AUS_IS_AIR_PSGR,AUS_IS_AIR_DPRT,AUS_ER_H2O_FWDM_ZS,AUS_ER_H2O_FWAG_ZS,AUS_EG_USE_COMM_CL_ZS,AUS_ER_H2O_FWTL_ZS,AUS_ER_H2O_FWIN_ZS,AUS_SH_HIV_ARTC_ZS,AUS_ER_H2O_FWTL_K3,AUS_AG_LND_ARBL_ZS,AUS_AG_LND_ARBL_HA,AUS_MS_MIL_TOTL_TF_ZS,AUS_AG_LND_ARBL_HA_PC,AUS_MS_MIL_TOTL_P1,AUS_MS_MIL_XPRT_KD,AUS_AG_LND_PRCP_MM,AUS_FB_ATM_TOTL_P5,AUS_MS_MIL_MPRT_KD,AUS_FD_RES_LIQU_AS_ZS,AUS_FB_BNK_CAPA_ZS,AUS_FB_AST_NPER_ZS,AUS_TM_TAX_TCOM_BC_ZS,AUS_TM_TAX_MRCH_BC_ZS,AUS_TM_TAX_MANF_BC_ZS,AUS_SH_STA_BRTC_ZS,AUS_SP_DYN_CBRT_IN,AUS_EN_BIR_THRD_NO,AUS_TM_TAX_MRCH_BR_ZS,CPI
0,1996-12-31,11.83,2.5,62.896067,3.318390e+11,10.648952,NaN,14.468028,NaN,NaN,NaN,21.867744,1.153740e+11,35645886.0,2129.5,3.128000e+09,3.376000e+09,2.471470e+09,2.561457e+09,7.920955,100.0,50477.0,41121.0,91598.0,3.223556,9.255749,NaN,NaN,10.745292,0.751036,0.422533,0.529255,302164.467,16.501800,56.097891,162.79,12.740137,38496.166,93849.531,31.059089,...,7.485202,48290.991768,1.422330,36729.629296,3.392160,21.145598,1.789700e+10,1.632010e+10,2.626800e+10,1833.699951,1.357891e+10,30075100.0,371200.0,NaN,69.85,1.229026,NaN,NaN,NaN,22.03,4.723065,36284000.0,0.642359,1.981541,58800.0,16000000.0,NaN,NaN,597000000.0,NaN,NaN,NaN,99.04,97.4,96.91,99.3,13.9,NaN,9.69,86.0
1,1997-12-31,11.83,2.5,64.128246,3.560984e+11,7.310580,NaN,15.844707,NaN,NaN,NaN,20.911558,1.161200e+11,31238927.0,1953.8,5.427000e+09,4.547000e+09,2.873247e+09,3.558460e+09,8.815737,100.0,56161.0,43284.0,99445.0,-5.047434,5.777851,NaN,NaN,12.378673,0.720054,0.411296,0.515179,305794.797,16.514273,56.139078,167.46,12.677627,38767.524,89434.463,29.246561,...,8.064125,50949.180512,1.337462,40196.161472,3.243344,8.082077,1.801000e+10,1.763910e+10,2.839100e+10,1953.800049,1.409454e+10,30953500.0,394500.0,NaN,69.85,1.283874,4.477642,NaN,NaN,22.19,5.189982,39871000.0,0.633353,2.153211,58400.0,20000000.0,534.0,NaN,35000000.0,NaN,NaN,NaN,99.04,97.4,96.91,99.3,13.6,NaN,9.69,88.6
2,1998-12-31,11.83,2.5,65.655622,3.861187e+11,8.430358,NaN,17.031299,NaN,NaN,NaN,16.910306,9.944900e+10,33339400.0,1986.3,4.025000e+09,2.842000e+09,-2.040032e+09,1.927694e+09,8.226741,100.0,60412.0,40697.0,101109.0,0.523480,5.772515,NaN,NaN,13.164531,0.699917,0.407558,0.510497,316883.805,16.935696,58.804818,186.30,12.731586,40344.334,87193.926,27.516056,...,6.996186,50567.629598,1.245301,40439.784391,3.038954,0.549470,1.787200e+10,1.773602e+10,2.854700e+10,1904.400024,1.212236e+10,30185700.0,378200.0,NaN,NaN,1.170228,NaN,NaN,NaN,NaN,5.734611,44055000.0,0.626453,2.354497,58400.0,4000000.0,NaN,NaN,542000000.0,NaN,NaN,0.7,99.04,97.4,96.91,99.3,13.3,NaN,9.69,87.0
3,1999-12-31,11.83,2.5,69.588639,4.313111e+11,11.704258,NaN,12.310128,NaN,0.253449,1.572000e+09,31.166616,1.931710e+11,35368800.0,2111.0,7.670000e+09,4.783000e+09,6.705487e+09,2.997243e+09,8.408477,100.0,67200.0,41246.0,108446.0,-2.873180,3.687329,NaN,NaN,14.634084,0.680790,0.398444,0.499082,325343.574,17.190298,59.058566,194.29,12.924641,42049.489,89148.437,27.401321,...,6.336334,56869.51100

In [0]:
import datetime
def datetiem(x):
    date_time_obj = datetime.datetime.strptime(x, '%Y-%m-%d' )
    return date_time_obj

In [0]:
df["Date"] = df["Date"].apply(lambda x: datetiem(x))
df.index = df.Date # index as date because time is not a variable
df=df.drop(['Date'], axis=1)

In [0]:
# df0 = df.fillna(0)
# removes columns where more than 30% of data is missing
# rationale for 0.7, if we can train test split for 0.7/ 0.3 we can accept at least 70% filled data
# fills remaining missing values with the mean of the column   
df_mean = df.loc[:, pd.notnull(df).sum()>len(df)*.70].fillna(df.mean())
# df_mean = df.loc[:, pd.notnull(df).sum()>len(df)*.70].fillna(df.interpolate())
df_mean.head()

,AUS_TM_TAX_MANF_BR_ZS,AUS_TM_TAX_TCOM_BR_ZS,AUS_FM_LBL_BMNY_GD_ZS,AUS_FM_LBL_BMNY_CN,AUS_FM_LBL_BMNY_ZG,AUS_FM_LBL_BMNY_IR_ZS,AUS_GC_BAL_CASH_GD_ZS,AUS_GC_BAL_CASH_CN,AUS_GC_DOD_TOTL_GD_ZS,AUS_GC_DOD_TOTL_CN,AUS_AG_PRD_CREL_MT,AUS_AG_YLD_CREL_KG,AUS_NE_GDI_STKB_KN,AUS_NE_GDI_STKB_CN,AUS_BN_RES_INCL_CD,AUS_NE_GDI_STKB_CD,AUS_NV_MNF_CHEM_ZS_UN,AUS_SH_HIV_0014,AUS_SE_PRM_UNER_MA,AUS_SE_PRM_UNER_FE,AUS_SE_PRM_UNER,AUS_FM_AST_CGOV_ZG_M3,AUS_FS_AST_CGOV_GD_ZS,AUS_FM_AST_DOMO_ZG_M3,AUS_FS_AST_DOMO_GD_ZS,AUS_FM_AST_PRVT_ZG_M3,AUS_EN_ATM_CO2E_PP_GD,AUS_EN_ATM_CO2E_KD_GD,AUS_EN_ATM_CO2E_PP_GD_KD,AUS_EN_ATM_CO2E_KT,AUS_EN_ATM_CO2E_PC,AUS_EN_CO2_ETOT_ZS,AUS_EN_CO2_ETOT_MT,AUS_EN_ATM_CO2E_GF_ZS,AUS_EN_ATM_CO2E_GF_KT,AUS_EN_ATM_CO2E_LF_KT,AUS_EN_ATM_CO2E_LF_ZS,AUS_EN_CO2_MANF_ZS,AUS_EN_CO2_MANF_MT,AUS_EN_CO2_OTHX_MT,...,AUS_NY_ADJ_DPEM_GN_ZS,AUS_NY_ADJ_DPEM_CD,AUS_SP_ADO_TFRT,AUS_SP_POP_DPND_OL,AUS_SP_POP_DPND,AUS_AG_LND_AGRI_ZS,AUS_SP_POP_DPND_YG,AUS_AG_LND_IRIG_AG_ZS,AUS_AG_LND_AGRI_K2,AUS_TX_VAL_AGRI_ZS_UN,AUS_TM_VAL_AGRI_ZS_UN,AUS_EA_PRD_AGRI_KD,AUS_NV_AGR_TOTL_ZS,AUS_NV_AGR_TOTL_KD_ZG,AUS_NV_AGR_TOTL_CN,AUS_NV_AGR_TOTL_KD,AUS_NV_AGR_TOTL_KN,AUS_IS_AIR_GOOD_MT_K1,AUS_NV_AGR_TOTL_CD,AUS_IS_AIR_PSGR,AUS_IS_AIR_DPRT,AUS_EG_USE_COMM_CL_ZS,AUS_SH_HIV_ARTC_ZS,AUS_AG_LND_ARBL_ZS,AUS_AG_LND_ARBL_HA,AUS_MS_MIL_TOTL_TF_ZS,AUS_AG_LND_ARBL_HA_PC,AUS_MS_MIL_TOTL_P1,AUS_MS_MIL_XPRT_KD,AUS_MS_MIL_MPRT_KD,AUS_FD_RES_LIQU_AS_ZS,AUS_FB_BNK_CAPA_ZS,AUS_FB_AST_NPER_ZS,AUS_TM_TAX_TCOM_BC_ZS,AUS_TM_TAX_MRCH_BC_ZS,AUS_TM_TAX_MANF_BC_ZS,AUS_SH_STA_BRTC_ZS,AUS_SP_DYN_CBRT_IN,AUS_TM_TAX_MRCH_BR_ZS,CPI
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1996-12-31,11.83,2.5,62.896067,3.318390e+11,10.648952,14.468028,-0.593380,-1.050719e+10,21.867744,1.153740e+11,35645886.0,2129.5,3.128000e+09,3.376000e+09,2.471470e+09,2.561457e+09,7.920955,100.0,50477.0,41121.0,91598.0,3.223556,9.255749,15.987804,139.465515,10.745292,0.751036,0.422533,0.529255,302164.467,16.501800,56.097891,162.79,12.740137,38496.166,93849.531,31.059089,15.538805,49.75,3.79,...,0.062109,2.397391e+08,19.7374,18.006447,50.276679,60.557515,32.270232,0.529334,4652210.0,7.485202,1.422330,36729.629296,3.392160,21.145598,1.789700e+10,1.632010e+10,2.626800e+10,1833.699951,1.357891e+10,30075100.0,371200.0,1.229026,60.411765,4.723065,36284000.0,0.642359,1.981541,58800.0,1.600000e+07,597000000.0,0.875655,5.905033,0.978065,99.04,97.4,96.91,99.3,13.9,9.69,86.0
1997-12-31,11.83,2.5,64.128246,3.560984e+11,7.310580,15.844707,-0.593380,-1.050719e+10,20.911558,1.161200e+11,31238927.0,1953.8,5.427000e+09,4.547000e+09,2.873247e+09,3.558460e+09,8.815737,100.0,56161.0,43284.0,99445.0,-5.047434,5.777851,15.987804,139.465515,12.378673,0.720054,0.411296,0.515179,305794.797,16.514273,56.139078,167.46,12.677627,38767.524,89434.463,29.246561,15.652834,51.12,3.94,...,0.061637,2.588741e+08,19.4530,18.137680,50.179157,60.164274,32.041478,0.529334,4622000.0,8.064125,1.337462,40196.161472,3.243344,8.082077,1.801000e+10,1.763910e+10,2.839100e+10,1953.800049,1.409454e+10,30953500.0,394500.0,1.283874,60.411765,5.189982,39871000.0,0.633353,2.153211,58400.0,2.000000e+07,35000000.0,0.875655,5.905033,0.978065,99.04,97.4,96.91,99.3,13.6,9.69,88.6
1998-12-31,11.83,2.5,65.655622,3.861187e+11,8.430358,17.031299,-0.593380,-1.050719e+10,16.910306,9.944900e+10,33339400.0,1986.3,4.025000e+09,2.842000e+09,-2.040032e+09,1.927694e+09,8.226741,100.0,60412.0,40697.0,101109.0,0.523480,5.772515,15.987804,139.465515,13.164531,0.699917,0.407558,0.510497,316883.805,16.935696,58.804818,186.30,12.731586,40344.334,87193.926,27.516056,14.691167,51.73,4.03,...,0.059302,2.289046e+08,18.9462,18.244330,49.999784,60.370722,31.755454,0.529334,4637860.0,6.996186,1.245301,40439.784391,3.038954,0.549470,1.787200e+10,1.773602e+10,2.854700e+10,1904.400024,1.212236e+10,30185700.0,378200.0,1.170228,60.411765,5.734611,44055000.0,0.626453,2.354497,58400.0,4.000000e+06,542000000.0,0.875655,5.905033,0.700000,99.04,97.4,96.91,99.3,13.3,9.69

In [0]:
CPI_list = list(df_mean.CPI)
def pct_df(df):
    df.drop(["CPI"], axis=1)
    new_df = df.pct_change()
    new_df.replace(np.nan, 0)
    new_df = new_df.replace([np.inf, -np.inf], 0)
    new_df = new_df.dropna(how='all')
    new_df['CPI']=CPI_list[1:]
    new_df = new_df.fillna(df.mean())
    return new_df
def UpsampleByTimeIndex(df,bytime,method):
    upsampled = df.resample(bytime)
    interpolated = upsampled.interpolate(method=method)
    return interpolated
# Ignore the index and treat the values as equally space
# ‘nearest’, ‘zero’, ‘slinear’, ‘quadratic’, ‘cubic’, ‘spline’, ‘barycentric’, ‘polynomial’
# .agg aggregates/ groups by specified time frame
# D for days, M for months
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.resample.html 
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.interpolate.html 
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.resample.Resampler.aggregate.html 
# https://machinelearningmastery.com/resample-interpolate-time-series-data-python/

In [0]:
pct_df_mean = pct_df(df_mean)
pct_df_mean

,AUS_TM_TAX_MANF_BR_ZS,AUS_TM_TAX_TCOM_BR_ZS,AUS_FM_LBL_BMNY_GD_ZS,AUS_FM_LBL_BMNY_CN,AUS_FM_LBL_BMNY_ZG,AUS_FM_LBL_BMNY_IR_ZS,AUS_GC_BAL_CASH_GD_ZS,AUS_GC_BAL_CASH_CN,AUS_GC_DOD_TOTL_GD_ZS,AUS_GC_DOD_TOTL_CN,AUS_AG_PRD_CREL_MT,AUS_AG_YLD_CREL_KG,AUS_NE_GDI_STKB_KN,AUS_NE_GDI_STKB_CN,AUS_BN_RES_INCL_CD,AUS_NE_GDI_STKB_CD,AUS_NV_MNF_CHEM_ZS_UN,AUS_SH_HIV_0014,AUS_SE_PRM_UNER_MA,AUS_SE_PRM_UNER_FE,AUS_SE_PRM_UNER,AUS_FM_AST_CGOV_ZG_M3,AUS_FS_AST_CGOV_GD_ZS,AUS_FM_AST_DOMO_ZG_M3,AUS_FS_AST_DOMO_GD_ZS,AUS_FM_AST_PRVT_ZG_M3,AUS_EN_ATM_CO2E_PP_GD,AUS_EN_ATM_CO2E_KD_GD,AUS_EN_ATM_CO2E_PP_GD_KD,AUS_EN_ATM_CO2E_KT,AUS_EN_ATM_CO2E_PC,AUS_EN_CO2_ETOT_ZS,AUS_EN_CO2_ETOT_MT,AUS_EN_ATM_CO2E_GF_ZS,AUS_EN_ATM_CO2E_GF_KT,AUS_EN_ATM_CO2E_LF_KT,AUS_EN_ATM_CO2E_LF_ZS,AUS_EN_CO2_MANF_ZS,AUS_EN_CO2_MANF_MT,AUS_EN_CO2_OTHX_MT,...,AUS_NY_ADJ_DPEM_GN_ZS,AUS_NY_ADJ_DPEM_CD,AUS_SP_ADO_TFRT,AUS_SP_POP_DPND_OL,AUS_SP_POP_DPND,AUS_AG_LND_AGRI_ZS,AUS_SP_POP_DPND_YG,AUS_AG_LND_IRIG_AG_ZS,AUS_AG_LND_AGRI_K2,AUS_TX_VAL_AGRI_ZS_UN,AUS_TM_VAL_AGRI_ZS_UN,AUS_EA_PRD_AGRI_KD,AUS_NV_AGR_TOTL_ZS,AUS_NV_AGR_TOTL_KD_ZG,AUS_NV_AGR_TOTL_CN,AUS_NV_AGR_TOTL_KD,AUS_NV_AGR_TOTL_KN,AUS_IS_AIR_GOOD_MT_K1,AUS_NV_AGR_TOTL_CD,AUS_IS_AIR_PSGR,AUS_IS_AIR_DPRT,AUS_EG_USE_COMM_CL_ZS,AUS_SH_HIV_ARTC_ZS,AUS_AG_LND_ARBL_ZS,AUS_AG_LND_ARBL_HA,AUS_MS_MIL_TOTL_TF_ZS,AUS_AG_LND_ARBL_HA_PC,AUS_MS_MIL_TOTL_P1,AUS_MS_MIL_XPRT_KD,AUS_MS_MIL_MPRT_KD,AUS_FD_RES_LIQU_AS_ZS,AUS_FB_BNK_CAPA_ZS,AUS_FB_AST_NPER_ZS,AUS_TM_TAX_TCOM_BC_ZS,AUS_TM_TAX_MRCH_BC_ZS,AUS_TM_TAX_MANF_BC_ZS,AUS_SH_STA_BRTC_ZS,AUS_SP_DYN_CBRT_IN,AUS_TM_TAX_MRCH_BR_ZS,CPI
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1997-12-31,0.000000,0.000000,0.019591,0.073106,-0.313493,0.095153,0.000000,0.000000,-0.043726,0.006466,-0.123632,-0.082508,0.734974,0.346860,0.162566,0.389233,0.112964,0.0,0.112606,0.052601,0.085668,-2.565797,-0.375755,0.000000,0.000000,0.152009,-0.041253,-0.026595,-0.026595,0.012014,0.000756,0.000734,0.028687,-0.004907,0.007049,-0.047044,-0.058357,0.007338,0.027538,0.039578,...,-0.007598,0.079816,-0.014409,0.007288,-0.001940,-0.006494,-0.007089,0.000000,-0.006494,0.077342,-0.059668,0.094380,-0.043870,-0.617789,0.006314,0.080821,0.080821,0.065496,0.037973,0.029207,0.062769,0.044628,0.000000,0.098859,0.098859,-0.014019,0.086634,-0.006803,0.250000,-0.941374,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,-0.021583,0.000000,88.6
1998-12-31,0.000000,0.000000,0.023818,0.084304,0.153172,0.074889,0.000000,0.000000,-0.191342,-0.143567,0.067239,0.016634,-0.258338,-0.374973,-1.710009,-0.458278,-0.066812,0.0,0.075693,-0.059768,0.016733,-1.103712,-0.000923,0.000000,0.000000,0.063485,-0.027965,-0.009089,-0.009089,0.036263,0.025519,0.047485,0.112504,0.004256,0.040673,-0.025052,-0.059170,-0.061437,0.011933,0.022843,...,-0.037883,-0.115769,-0.026053,0.005880,-0.003575,0.003431,-0.008927,0.000000,0.003431,-0.132431,-0.068907,0.006061,-0.063018,-0.932014,-0.007662,0.005495,0.005495,-0.025284,-0.139925,-0.024805,-0.041318,-0.088518,0.000000,0.104938,0.104938,-0.010895,0.093482,0.000000,-0.800000,14.485714,0.000000,0.000000,-0.284301,0.000000,0.000000,0.000000,0.000000e+00,-0.022059,0.000000,87.0
1999-12-31,0.000000,0.000000,0.059904,0.117043,0.388347,-0.277206,-1.427128,-1.149612,0.843055,0.942413,0.060871,0.062780,0.905590,0.682970,-4.286953,0.554833,0.022091,0.0,0.112362,0.013490,0.072565,-6.488620,-0.361227,0.000000,0.000000,0.111630,-0.027327,-0.022362,-0.022362,0.026697,0.015033,0.004315,0.042888,0.015163,0.042265,0.022416,-0.004170,-0.018077,0.001933,0.017370,...,-0.060136,-0.084485,-0.026749,0.005916,-0.003320,-0.021685,-0.008627,0.000000,-0.021685,-0.094316,-0.015795,0.104574,0.007939,17.736791,0.062276,0.102953,0.102953,-0.111006,-0.018603,0.046181,0.017451,0.023315,0.000000,0.033481,0.033481,-0.047297,0.021740,-0.037671,12.644737,0.241697,0.000000,0.000000,-0.142857,0.000000,0.000000,0.000000,0.000000e+00,-0.015038,0.000000,87.0
2000-12-31,0.000000,0.000000,-0.027053,0.

In [0]:
#Ignore this code; helper functions for binarizing
list_col_names = list(pct_df_mean.columns.values)
from itertools import repeat
def binarize(x):
    if x< 0:
        x = -1
    elif x>0:
        x=1
    else:
        x = 0
    return x
month_cpi = [x for item in CPI_list[1:len(CPI_list)-1] for x in repeat(item, 12)]
month_cpi.insert(len(month_cpi),CPI_list[len(CPI_list)-1])
def bin_action_cpi(df):
    df.drop(["CPI"], axis=1)
    df = df.applymap(binarize)
    try:
        df['CPI']=CPI_list[1:]
    except:
        df['CPI']= month_cpi
    return df
def bin_action(df):
    df = df.applymap(binarize)
    return df
def binarize10(x):
    if x<= 0:
        x = 0
    else:
        x = 1
    return x
def bin_action_cpi10(df):
    df.drop(["CPI"], axis=1)
    df = df.applymap(binarize10)
    try:
        df['CPI']=CPI_list[1:]
    except:
        df['CPI']= month_cpi
    return df
def bin_action10(df):
    df = df.applymap(binarize10)
    return df
def binarize123(x):
    if x< 0:
        x = 1
    elif x>0:
        x=3
    else:
        x = 2
    return x
def bin_action_cpi123(df):
    df.drop(["CPI"], axis=1)
    df = df.applymap(binarize123)
    try:
        df['CPI']=CPI_list[1:]
    except:
        df['CPI']= month_cpi
    return df
def bin_action123(df):
    df = df.applymap(binarize123)
    return df

In [0]:
# UpsampleByTimeIndex(df_mean,'M','linear')
bin_action_cpi123(df_latest)

,AUS_TM_TAX_MANF_BR_ZS,AUS_TM_TAX_TCOM_BR_ZS,AUS_FM_LBL_BMNY_GD_ZS,AUS_FM_LBL_BMNY_CN,AUS_FM_LBL_BMNY_ZG,AUS_FM_LBL_BMNY_IR_ZS,AUS_GC_BAL_CASH_GD_ZS,AUS_GC_BAL_CASH_CN,AUS_GC_DOD_TOTL_GD_ZS,AUS_GC_DOD_TOTL_CN,AUS_AG_PRD_CREL_MT,AUS_AG_YLD_CREL_KG,AUS_NE_GDI_STKB_KN,AUS_NE_GDI_STKB_CN,AUS_BN_RES_INCL_CD,AUS_NE_GDI_STKB_CD,AUS_NV_MNF_CHEM_ZS_UN,AUS_SH_HIV_0014,AUS_SE_PRM_UNER_MA,AUS_SE_PRM_UNER_FE,AUS_SE_PRM_UNER,AUS_FM_AST_CGOV_ZG_M3,AUS_FS_AST_CGOV_GD_ZS,AUS_FM_AST_DOMO_ZG_M3,AUS_FS_AST_DOMO_GD_ZS,AUS_FM_AST_PRVT_ZG_M3,AUS_EN_ATM_CO2E_PP_GD,AUS_EN_ATM_CO2E_KD_GD,AUS_EN_ATM_CO2E_PP_GD_KD,AUS_EN_ATM_CO2E_KT,AUS_EN_ATM_CO2E_PC,AUS_EN_CO2_ETOT_ZS,AUS_EN_CO2_ETOT_MT,AUS_EN_ATM_CO2E_GF_ZS,AUS_EN_ATM_CO2E_GF_KT,AUS_EN_ATM_CO2E_LF_KT,AUS_EN_ATM_CO2E_LF_ZS,AUS_EN_CO2_MANF_ZS,AUS_EN_CO2_MANF_MT,AUS_EN_CO2_OTHX_MT,...,AUS_NY_ADJ_DPEM_GN_ZS,AUS_NY_ADJ_DPEM_CD,AUS_SP_ADO_TFRT,AUS_SP_POP_DPND_OL,AUS_SP_POP_DPND,AUS_AG_LND_AGRI_ZS,AUS_SP_POP_DPND_YG,AUS_AG_LND_IRIG_AG_ZS,AUS_AG_LND_AGRI_K2,AUS_TX_VAL_AGRI_ZS_UN,AUS_TM_VAL_AGRI_ZS_UN,AUS_EA_PRD_AGRI_KD,AUS_NV_AGR_TOTL_ZS,AUS_NV_AGR_TOTL_KD_ZG,AUS_NV_AGR_TOTL_CN,AUS_NV_AGR_TOTL_KD,AUS_NV_AGR_TOTL_KN,AUS_IS_AIR_GOOD_MT_K1,AUS_NV_AGR_TOTL_CD,AUS_IS_AIR_PSGR,AUS_IS_AIR_DPRT,AUS_EG_USE_COMM_CL_ZS,AUS_SH_HIV_ARTC_ZS,AUS_AG_LND_ARBL_ZS,AUS_AG_LND_ARBL_HA,AUS_MS_MIL_TOTL_TF_ZS,AUS_AG_LND_ARBL_HA_PC,AUS_MS_MIL_TOTL_P1,AUS_MS_MIL_XPRT_KD,AUS_MS_MIL_MPRT_KD,AUS_FD_RES_LIQU_AS_ZS,AUS_FB_BNK_CAPA_ZS,AUS_FB_AST_NPER_ZS,AUS_TM_TAX_TCOM_BC_ZS,AUS_TM_TAX_MRCH_BC_ZS,AUS_TM_TAX_MANF_BC_ZS,AUS_SH_STA_BRTC_ZS,AUS_SP_DYN_CBRT_IN,AUS_TM_TAX_MRCH_BR_ZS,CPI
0,2,2,3,3,1,3,2,2,1,3,1,1,3,3,3,3,3,2,3,3,3,1,1,2,2,3,1,1,1,3,3,3,3,1,3,1,1,3,3,3,...,1,3,1,3,1,1,1,2,1,3,1,3,1,1,3,3,3,3,3,3,3,3,2,3,3,1,3,1,3,1,2,2,2,2,2,2,2,1,2,88.6
1,2,2,3,3,1,3,2,2,1,1,1,1,3,3,3,3,3,2,3,3,3,1,1,2,2,3,1,1,1,3,3,3,3,1,3,1,1,3,3,3,...,1,3,1,3,1,1,1,2,1,3,1,3,1,1,3,3,3,3,3,3,3,3,2,3,3,1,3,1,3,3,2,2,1,2,2,2,2,1,2,88.6
2,2,2,3,3,1,3,2,2,1,1,1,1,3,3,1,3,3,2,3,3,3,1,1,2,2,3,1,1,1,3,3,3,3,1,3,1,1,1,3,3,...,1,3,1,3,1,1,1,2,1,3,1,3,1,1,3,3,3,3,3,3,3,3,2,3,3,1,3,1,3,3,2,2,1,2,2,2,2,1,2,88.6
3,2,2,3,3,1,3,2,2,1,1,1,1,3,3,1,3,3,2,3,3,3,1,1,2,2,3,1,1,1,3,3,3,3,1,3,1,1,1,3,3,...,1,3,1,3,1,1,1,2,1,3,1,3,1,1,3,3,3,3,1,3,3,3,2,3,3,1,3,1,1,3,2,2,1,2,2,2,2,1,2,88.6
4,2,2,3,3,1,3,2,2,1,1,1,1,3,3,1,3,3,2,3,3,3,1,1,2,2,3,1,1,1,3,3,3,3,1,3,1,1,1,3,3,...,1,3,1,3,1,1,1,2,1,3,1,3,1,1,3,3,3,3,1,3,3,3,2,3,3,1,3,1,1,3,2,2,1,2,2,2,2,1,2,88.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,1,1,3,3,3,1,1,1,3,3,1,1,1,1,1,1,3,2,3,3,3,3,3,1,3,1,3,3,3,1,3,3,2,1,1,1,1,3,2,2,...,1,1,1,3,3,1,3,1,1,3,3,1,3,1,3,1,1,1,1,3,3,1,3,1,1,1,1,3,3,3,1,3,3,2,3,3,1,1,1,79.0
225,1,1,3,3,3,1,1,1,3,3,1,3,1,1,1,1,3,2,3,3,3,3,3,1,3,1,3,3,3,1,3,3,2,1,1,1,1,3,2,2,...,1,1,1,3,3,1,3,1,1,3,3,1,3,1,3,1,1,1,1,3,3,1,3,1,1,1,1,3,3,1,1,3,3,2,3,3,1,1,1,79.0
226,1,1,3,3,3,1,1,1,3,3,1,3,1,1,1,1,3,2,3,3,3,3,3,1,3,1,3,3,3,1,3,3,2,1,1,1,1,3,2,2,...,1,1,1,3,3,1,3,1,1,3,3,1,3,1,3,1,1,1,1,3,3,1,3,1,1,1,1,3,3,1,1,3,3,2,3,3,1,1,1,79.0
227,1,1,3,3,3,1,1,1,3,3,1,3,1,1,1,1,3,2,3,3,3,3,3,1,3,1,3,3,3,1,3,3,2,1,1,1,1,3,2,2,...,1,1,1,3,3,3,3,1,3,3,1,1,3,1,3,1,1,1,1,3,3,1,3,1,1,1,1,3,3,1,1,3,3,2,3,3,1,1,1,79.0


In [0]:
# percentage change df and binarize 123
# df_mean_wo_cpi = pct_df_mean.drop(["CPI"], axis=1) # remove cpi to create array of test data
# df_latest = UpsampleByTimeIndex(pct_df_mean,'Y','linear')
# df_latest = bin_action_cpi123(df_latest)
# df_latest.index = list(range(len(df_latest)))
# df_latest_wo_cpi = UpsampleByTimeIndex(df_mean_wo_cpi,'Y','linear')
# df_latest_wo_cpi = bin_action123(df_latest_wo_cpi)
# df_latest_wo_cpi.index = list(range(len(df_latest_wo_cpi)))

# percentage change df and binarize 0,1
# df_mean_wo_cpi = pct_df_mean.drop(["CPI"], axis=1) # remove cpi to create array of test data
# df_latest = UpsampleByTimeIndex(pct_df_mean,'M','linear')
# df_latest = bin_action_cpi10(df_latest)
# df_latest.index = list(range(len(df_latest)))
# df_latest_wo_cpi = UpsampleByTimeIndex(df_mean_wo_cpi,'M','linear')
# df_latest_wo_cpi = bin_action10(df_latest_wo_cpi)
# df_latest_wo_cpi.index = list(range(len(df_latest_wo_cpi)))

# percentage change df and binarize -1,0,1
# df_mean_wo_cpi = pct_df_mean.drop(["CPI"], axis=1) # remove cpi to create array of test data
# df_latest = UpsampleByTimeIndex(pct_df_mean,'Y','linear')
# df_latest = bin_action_cpi(df_latest)
# df_latest.index = list(range(len(df_latest)))
# df_latest_wo_cpi = UpsampleByTimeIndex(df_mean_wo_cpi,'Y','linear')
# df_latest_wo_cpi = bin_action(df_latest_wo_cpi)
# df_latest_wo_cpi.index = list(range(len(df_latest_wo_cpi)))

# percentage change df
df_mean_wo_cpi = pct_df_mean.drop(["CPI"], axis=1) # remove cpi to create array of test data
df_latest = UpsampleByTimeIndex(pct_df_mean,'M','linear')
df_latest.index = list(range(len(df_latest)))
df_latest_wo_cpi = UpsampleByTimeIndex(df_mean_wo_cpi,'M','linear')
df_latest_wo_cpi.index = list(range(len(df_latest_wo_cpi)))

# mean df
# df_mean_wo_cpi = df_mean.drop(["CPI"], axis=1) # remove cpi to create array of test data
# df_latest = UpsampleByTimeIndex(df_mean,'Y','linear')
# df_latest.index = list(range(len(df_latest)))
# df_latest_wo_cpi = UpsampleByTimeIndex(df_mean_wo_cpi,'Y','linear')
# df_latest_wo_cpi.index = list(range(len(df_latest_wo_cpi)))

In [0]:
df_latest_wo_cpi

,AUS_TM_TAX_MANF_BR_ZS,AUS_TM_TAX_TCOM_BR_ZS,AUS_FM_LBL_BMNY_GD_ZS,AUS_FM_LBL_BMNY_CN,AUS_FM_LBL_BMNY_ZG,AUS_FM_LBL_BMNY_IR_ZS,AUS_GC_BAL_CASH_GD_ZS,AUS_GC_BAL_CASH_CN,AUS_GC_DOD_TOTL_GD_ZS,AUS_GC_DOD_TOTL_CN,AUS_AG_PRD_CREL_MT,AUS_AG_YLD_CREL_KG,AUS_NE_GDI_STKB_KN,AUS_NE_GDI_STKB_CN,AUS_BN_RES_INCL_CD,AUS_NE_GDI_STKB_CD,AUS_NV_MNF_CHEM_ZS_UN,AUS_SH_HIV_0014,AUS_SE_PRM_UNER_MA,AUS_SE_PRM_UNER_FE,AUS_SE_PRM_UNER,AUS_FM_AST_CGOV_ZG_M3,AUS_FS_AST_CGOV_GD_ZS,AUS_FM_AST_DOMO_ZG_M3,AUS_FS_AST_DOMO_GD_ZS,AUS_FM_AST_PRVT_ZG_M3,AUS_EN_ATM_CO2E_PP_GD,AUS_EN_ATM_CO2E_KD_GD,AUS_EN_ATM_CO2E_PP_GD_KD,AUS_EN_ATM_CO2E_KT,AUS_EN_ATM_CO2E_PC,AUS_EN_CO2_ETOT_ZS,AUS_EN_CO2_ETOT_MT,AUS_EN_ATM_CO2E_GF_ZS,AUS_EN_ATM_CO2E_GF_KT,AUS_EN_ATM_CO2E_LF_KT,AUS_EN_ATM_CO2E_LF_ZS,AUS_EN_CO2_MANF_ZS,AUS_EN_CO2_MANF_MT,AUS_EN_CO2_OTHX_MT,...,AUS_NY_ADJ_NNAT_GN_ZS,AUS_NY_ADJ_DPEM_GN_ZS,AUS_NY_ADJ_DPEM_CD,AUS_SP_ADO_TFRT,AUS_SP_POP_DPND_OL,AUS_SP_POP_DPND,AUS_AG_LND_AGRI_ZS,AUS_SP_POP_DPND_YG,AUS_AG_LND_IRIG_AG_ZS,AUS_AG_LND_AGRI_K2,AUS_TX_VAL_AGRI_ZS_UN,AUS_TM_VAL_AGRI_ZS_UN,AUS_EA_PRD_AGRI_KD,AUS_NV_AGR_TOTL_ZS,AUS_NV_AGR_TOTL_KD_ZG,AUS_NV_AGR_TOTL_CN,AUS_NV_AGR_TOTL_KD,AUS_NV_AGR_TOTL_KN,AUS_IS_AIR_GOOD_MT_K1,AUS_NV_AGR_TOTL_CD,AUS_IS_AIR_PSGR,AUS_IS_AIR_DPRT,AUS_EG_USE_COMM_CL_ZS,AUS_SH_HIV_ARTC_ZS,AUS_AG_LND_ARBL_ZS,AUS_AG_LND_ARBL_HA,AUS_MS_MIL_TOTL_TF_ZS,AUS_AG_LND_ARBL_HA_PC,AUS_MS_MIL_TOTL_P1,AUS_MS_MIL_XPRT_KD,AUS_MS_MIL_MPRT_KD,AUS_FD_RES_LIQU_AS_ZS,AUS_FB_BNK_CAPA_ZS,AUS_FB_AST_NPER_ZS,AUS_TM_TAX_TCOM_BC_ZS,AUS_TM_TAX_MRCH_BC_ZS,AUS_TM_TAX_MANF_BC_ZS,AUS_SH_STA_BRTC_ZS,AUS_SP_DYN_CBRT_IN,AUS_TM_TAX_MRCH_BR_ZS
0,0.000000,0.000000,0.019591,0.073106,-0.313493,0.095153,0.000000,0.000000,-0.043726,0.006466,-0.123632,-0.082508,0.734974,0.346860,0.162566,0.389233,0.112964,0.0,0.112606,0.052601,0.085668,-2.565797,-0.375755,0.000000,0.000000,0.152009,-0.041253,-0.026595,-0.026595,0.012014,0.000756,0.000734,0.028687,-0.004907,0.007049,-0.047044,-0.058357,0.007338,0.027538,0.039578,...,0.126525,-0.007598,0.079816,-0.014409,0.007288,-0.001940,-0.006494,-0.007089,0.000000,-0.006494,0.077342,-0.059668,0.094380,-0.043870,-0.617789,0.006314,0.080821,0.080821,0.065496,0.037973,0.029207,0.062769,0.044628,0.000000,0.098859,0.098859,-0.014019,0.086634,-0.006803,0.250000,-0.941374,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,-0.021583,0.000000
1,0.000000,0.000000,0.019943,0.074039,-0.274604,0.093464,0.000000,0.000000,-0.056027,-0.006037,-0.107726,-0.074246,0.652198,0.286707,0.006518,0.318607,0.097983,0.0,0.109530,0.043237,0.079923,-2.443957,-0.344519,0.000000,0.000000,0.144632,-0.040146,-0.025136,-0.025136,0.014035,0.002819,0.004630,0.035672,-0.004143,0.009851,-0.045211,-0.058425,0.001607,0.026237,0.038183,...,0.111572,-0.010121,0.063517,-0.015379,0.007171,-0.002076,-0.005667,-0.007242,0.000000,-0.005667,0.059861,-0.060438,0.087020,-0.045466,-0.643975,0.005149,0.074544,0.074544,0.057931,0.023148,0.024706,0.054095,0.033532,0.000000,0.099366,0.099366,-0.013759,0.087205,-0.006236,0.162500,0.344217,0.000000,0.000000,-0.023692,0.0,0.000000,0.000000,0.000000,-0.021622,0.000000
2,0.000000,0.000000,0.020295,0.074972,-0.235715,0.091776,0.000000,0.000000,-0.068329,-0.018540,-0.091820,-0.065984,0.569422,0.226555,-0.149530,0.247981,0.083001,0.0,0.106454,0.033873,0.074179,-2.322116,-0.313283,0.000000,0.000000,0.137255,-0.039038,-0.023677,-0.023677,0.016056,0.004883,0.008526,0.042657,-0.003379,0.012653,-0.043379,-0.058493,-0.004124,0.024937,0.036789,...,0.096619,-0.012645,0.047219,-0.016350,0.007053,-0.002212,-0.004840,-0.007395,0.000000,-0.004840,0.042380,-0.061208,0.079660,-0.047062,-0.670160,0.003985,0.068266,0.068266,0.050366,0.008323,0.020205,0.045421,0.022437,0.000000,0.099872,0.099872,-0.013499,0.087776,-0.005669,0.075000,1.629808,0.000000,0.000000,-0.047384,0.0,0.000000,0.000000,0.000000,-0.021662,0.000000
3,0.000000,0.000000,0.020647,0.075905,-0.196827,0.090087,0.000000,0.000000,-0.080630,-0.031042,-0.075914,-0.057722,0.486646,0.166402,-0.305578,0.177355,0.068020,

In [0]:
# check for null, if false then no missing values
def null_check(dfr):
    return dfr.isnull().values.any()
# print(null_check(df0))
# print(null_check(df_mean))
print(null_check(df_latest_wo_cpi))

False


In [0]:
from sklearn.preprocessing import MinMaxScaler
sc= MinMaxScaler(feature_range=(0,1))

input_data = df_latest.values
test_data = df_latest_wo_cpi.values
# input_data[:,0:len(input_data[0])-1] = sc.fit_transform(input_data[:,0:len(input_data[0])-1])
# test_data[:,0:len(test_data[0])] = sc.fit_transform(test_data[:,:])
test_data

array([[ 0.        ,  0.        ,  0.01959072, ...,  0.        ,
        -0.02158273,  0.        ],
       [ 0.        ,  0.        ,  0.01994295, ...,  0.        ,
        -0.02162241,  0.        ],
       [ 0.        ,  0.        ,  0.02029519, ...,  0.        ,
        -0.02166208,  0.        ],
       ...,
       [-0.00312925, -0.00347222,  0.04227311, ..., -0.00334336,
        -0.00378788, -0.002079  ],
       [-0.00156463, -0.00173611,  0.04219153, ..., -0.0036777 ,
        -0.00189394, -0.0010395 ],
       [ 0.        ,  0.        ,  0.04210995, ..., -0.00401204,
         0.        ,  0.        ]])

In [0]:
print(test_data.shape)
print(input_data.shape) # check data and shape, are there 22/ 229 rows?

(229, 674)
(229, 675)


In [0]:
df_latest.head()

,AUS_TM_TAX_MANF_BR_ZS,AUS_TM_TAX_TCOM_BR_ZS,AUS_FM_LBL_BMNY_GD_ZS,AUS_FM_LBL_BMNY_CN,AUS_FM_LBL_BMNY_ZG,AUS_FM_LBL_BMNY_IR_ZS,AUS_GC_BAL_CASH_GD_ZS,AUS_GC_BAL_CASH_CN,AUS_GC_DOD_TOTL_GD_ZS,AUS_GC_DOD_TOTL_CN,AUS_AG_PRD_CREL_MT,AUS_AG_YLD_CREL_KG,AUS_NE_GDI_STKB_KN,AUS_NE_GDI_STKB_CN,AUS_BN_RES_INCL_CD,AUS_NE_GDI_STKB_CD,AUS_NV_MNF_CHEM_ZS_UN,AUS_SH_HIV_0014,AUS_SE_PRM_UNER_MA,AUS_SE_PRM_UNER_FE,AUS_SE_PRM_UNER,AUS_FM_AST_CGOV_ZG_M3,AUS_FS_AST_CGOV_GD_ZS,AUS_FM_AST_DOMO_ZG_M3,AUS_FS_AST_DOMO_GD_ZS,AUS_FM_AST_PRVT_ZG_M3,AUS_EN_ATM_CO2E_PP_GD,AUS_EN_ATM_CO2E_KD_GD,AUS_EN_ATM_CO2E_PP_GD_KD,AUS_EN_ATM_CO2E_KT,AUS_EN_ATM_CO2E_PC,AUS_EN_CO2_ETOT_ZS,AUS_EN_CO2_ETOT_MT,AUS_EN_ATM_CO2E_GF_ZS,AUS_EN_ATM_CO2E_GF_KT,AUS_EN_ATM_CO2E_LF_KT,AUS_EN_ATM_CO2E_LF_ZS,AUS_EN_CO2_MANF_ZS,AUS_EN_CO2_MANF_MT,AUS_EN_CO2_OTHX_MT,...,AUS_NY_ADJ_DPEM_GN_ZS,AUS_NY_ADJ_DPEM_CD,AUS_SP_ADO_TFRT,AUS_SP_POP_DPND_OL,AUS_SP_POP_DPND,AUS_AG_LND_AGRI_ZS,AUS_SP_POP_DPND_YG,AUS_AG_LND_IRIG_AG_ZS,AUS_AG_LND_AGRI_K2,AUS_TX_VAL_AGRI_ZS_UN,AUS_TM_VAL_AGRI_ZS_UN,AUS_EA_PRD_AGRI_KD,AUS_NV_AGR_TOTL_ZS,AUS_NV_AGR_TOTL_KD_ZG,AUS_NV_AGR_TOTL_CN,AUS_NV_AGR_TOTL_KD,AUS_NV_AGR_TOTL_KN,AUS_IS_AIR_GOOD_MT_K1,AUS_NV_AGR_TOTL_CD,AUS_IS_AIR_PSGR,AUS_IS_AIR_DPRT,AUS_EG_USE_COMM_CL_ZS,AUS_SH_HIV_ARTC_ZS,AUS_AG_LND_ARBL_ZS,AUS_AG_LND_ARBL_HA,AUS_MS_MIL_TOTL_TF_ZS,AUS_AG_LND_ARBL_HA_PC,AUS_MS_MIL_TOTL_P1,AUS_MS_MIL_XPRT_KD,AUS_MS_MIL_MPRT_KD,AUS_FD_RES_LIQU_AS_ZS,AUS_FB_BNK_CAPA_ZS,AUS_FB_AST_NPER_ZS,AUS_TM_TAX_TCOM_BC_ZS,AUS_TM_TAX_MRCH_BC_ZS,AUS_TM_TAX_MANF_BC_ZS,AUS_SH_STA_BRTC_ZS,AUS_SP_DYN_CBRT_IN,AUS_TM_TAX_MRCH_BR_ZS,CPI
0,0.320921,0.618753,0.458239,0.273074,0.167647,0.227845,0.342854,0.336228,0.142707,0.138154,0.221479,0.281548,0.960679,0.704864,0.080047,0.701291,0.902639,0.0,0.861909,0.679171,0.792372,0.612094,0.565072,0.133608,0.602079,0.600038,0.104065,0.113575,0.113575,0.576735,0.341774,0.390492,0.594116,0.582669,0.555698,0.404683,0.367183,0.602365,0.601066,0.410329,...,0.713996,0.567026,0.487162,0.183388,0.232583,0.730080,0.292705,0.6271,0.730080,0.655566,0.408188,0.651454,0.567941,0.282535,0.535162,0.629980,0.629980,0.502984,0.334515,0.239416,0.556577,0.659633,0.679796,0.891345,0.891345,0.530201,0.903378,0.468224,0.049257,0.000000,0.256644,0.600738,0.154799,0.29983,0.7291,0.888428,0.498995,0.062300,0.478267,88.600000
1,0.320921,0.618753,0.461161,0.279284,0.180742,0.226941,0.342854,0.336228,0.130815,0.126642,0.229529,0.286683,0.957726,0.697766,0.077432,0.694255,0.836000,0.0,0.853819,0.656300,0.777443,0.625132,0.567383,0.133608,0.602079,0.592163,0.105517,0.117831,0.117831,0.598775,0.353937,0.441284,0.627940,0.584513,0.561583,0.411377,0.366872,0.586911,0.597985,0.407295,...,0.691629,0.528600,0.457094,0.178707,0.226490,0.736584,0.285760,0.6271,0.736584,0.626857,0.405971,0.635763,0.563904,0.281511,0.532445,0.616658,0.616658,0.491758,0.306639,0.220704,0.545832,0.641145,0.679796,0.893943,0.893943,0.531813,0.906449,0.471761,0.045678,0.083333,0.256644,0.600738,0.143796,0.29983,0.7291,0.888428,0.498995,0.061976,0.478267,88.466667
2,0.320921,0.618753,0.464083,0.285495,0.193837,0.226037,0.342854,0.336228,0.118923,0.115129,0.237579,0.291818,0.954773,0.690667,0.074818,0.687219,0.769362,0.0,0.845728,0.633430,0.762514,0.638171,0.569693,0.133608,0.602079,0.584287,0.106970,0.122086,0.122086,0.620815,0.366100,0.492077,0.661763,0.586356,0.567469,0.418072,0.366560,0.571457,0.594905,0.404261,...,0.669262,0.490174,0.427026,0.174026,0.220398,0.743088,0.278816,0.6271,0.743088,0.598147,0.403754,0.620072,0.559867,0.280487,0.529727,0.603337,0.603337,0.480532,0.278762,0.201993,0.535086,0.622657,0.679796,0.896542,0.896542,0.533424,0.909521,0.475298,0.042100,0.166667,0.256644,0.600738,0.132794,0.29983,0.7291,0.888428,0.498995,0.061651,0.478267,88.333333
3,0.320921,0.618753,0.467005,0.291705,0.206931,0.225133,0.342854,0.336228,0.107030,0.103616,0.245630,0.296952,0.951820,0.683569,0.072203,0.680183,0.702723,0.0,0.837638,0.610559,0.747585,0.651209,0.572004,0.133608,0.602079,0.576411,0.108423,0.126342,0.1263

In [0]:
n_steps = 3
df_test=test_data[-n_steps:]
CPI_test = df_latest['CPI'][-n_steps:]
print(CPI_test)
print(df_test.shape)
input_feature = input_data[:len(input_data)-n_steps] # to remove last 5 rows, follow the time step you set

226    79.0
227    79.0
228    79.0
Name: CPI, dtype: float64
(3, 674)


In [0]:
input_feature.shape # check shape 

(226, 675)

In [0]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

In [0]:
# choose a number of time steps
from numpy import array
# convert into input/output
X, y = split_sequences(input_feature, n_steps)
print(X.shape, y.shape)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])
# the array is (Total number of samples, time step, total number of dimensions)

(224, 3, 674) (224,)
[[0.3209214  0.61875297 0.45823875 ... 0.49899497 0.06230016 0.4782671 ]
 [0.3209214  0.61875297 0.46116087 ... 0.49899497 0.06197555 0.4782671 ]
 [0.3209214  0.61875297 0.464083   ... 0.49899497 0.06165094 0.4782671 ]] 88.33333333333333
[[0.3209214  0.61875297 0.46116087 ... 0.49899497 0.06197555 0.4782671 ]
 [0.3209214  0.61875297 0.464083   ... 0.49899497 0.06165094 0.4782671 ]
 [0.3209214  0.61875297 0.46700512 ... 0.49899497 0.06132632 0.4782671 ]] 88.19999999999999
[[0.3209214  0.61875297 0.464083   ... 0.49899497 0.06165094 0.4782671 ]
 [0.3209214  0.61875297 0.46700512 ... 0.49899497 0.06132632 0.4782671 ]
 [0.3209214  0.61875297 0.46992724 ... 0.49899497 0.06100171 0.4782671 ]] 88.06666666666666
[[0.3209214  0.61875297 0.46700512 ... 0.49899497 0.06132632 0.4782671 ]
 [0.3209214  0.61875297 0.46992724 ... 0.49899497 0.06100171 0.4782671 ]
 [0.3209214  0.61875297 0.47284937 ... 0.49899497 0.0606771  0.4782671 ]] 87.93333333333334
[[0.3209214  0.61875297 0.4

In [0]:
# final model
# https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/ 
# refer to this if necessary
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
model = Sequential()
model.add(LSTM(75, activation='relu', input_shape=(n_steps, X.shape[2])))
model.add(Dense(15))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# model.save_weights('my_model_weights.h5')

In [0]:
# fit model
model.fit(X, y, epochs=200, verbose=0)#, validation_split=0.2, batch_size=5)

In [0]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 75)                225000    
_________________________________________________________________
dense_3 (Dense)              (None, 15)                1140      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 16        
Total params: 226,156
Trainable params: 226,156
Non-trainable params: 0
_________________________________________________________________


In [0]:
# demonstrate prediction
x_input = df_test
x_input = x_input.reshape((1, n_steps,X.shape[2] ))
yhat = model.predict(x_input, verbose=0)

Percentage Error

In [0]:
print(yhat)
actual = CPI_test[len(df_latest)-1]
print(float((abs(actual-yhat)/actual)*100))

[[79.0546]]
0.0691184550523758


In [0]:
weight_tensors = model.layers[1].weights
weight_names = list(map(lambda x: x.name, weight_tensors))
print(weight_names)

['dense_3/kernel:0', 'dense_3/bias:0']


In [0]:
for layer in model.layers:
        if "LSTM" in str(layer):
            weightLSTM = layer.get_weights()
warr,uarr, barr = weightLSTM
print(warr.shape) # warr is a numpy array of weights for inputs
print(uarr.shape) # uarr is a numpy array of weights for hidden units
print(barr.shape) # barr is a numpy array of bias

(674, 300)
(75, 300)
(300,)


In [0]:
# print(len(warr[0]))
# print(len(uarr[0])) 
print(barr) 

[-0.00883811 -0.00716633  0.01842842 -0.00658883  0.          0.02010401
 -0.00834471  0.01865833  0.          0.02129459  0.01936749 -0.01003752
  0.01992659  0.01706819 -0.01043026  0.01700198 -0.01253282  0.01929402
 -0.00589997 -0.00738681 -0.00555864 -0.00698286  0.01093439 -0.00555492
 -0.00616567  0.01900841 -0.01386638 -0.00556287 -0.00748665  0.
  0.01378517 -0.00611462 -0.00235413  0.          0.01714375  0.02088303
 -0.0060831  -0.00555797  0.01200219  0.01892681 -0.00511063 -0.00578824
  0.01981456  0.01937706  0.01389886 -0.00711386  0.0158329   0.
 -0.00580852  0.01312262  0.          0.02112342 -0.00658557 -0.00518681
 -0.00642113  0.0165755   0.00284425 -0.00555781 -0.00947059 -0.00775055
  0.01623153 -0.00754853 -0.00555646 -0.00555135  0.00911162 -0.00848775
 -0.00555582 -0.00757073  0.01975223  0.         -0.0069811   0.01662062
 -0.00915623  0.00752452  0.01751046  0.9919241   0.9928448   1.0136542
  0.9929741   1.          1.0140535   0.99238306  1.0157626   1.
  1

In [0]:
# change df to percentage changes
# play with the rank/ totaln
# interpolate lesser 
# time step

In [0]:
time step = 5, CPI, indicator data
interpolate by year, 22 data points
# 20.441774368286133
interpolate by month, 262 DP
# 1.0591048002243042
interpolate by day, 7920 DP
# 0.015229816548526287 

After performing percentage change and normalizing 
interpolate by year time step 7
# 0.6501209735870361
# 351430.6875
interpolate by year time step 5
# 0.5464288592338562
# 320606.21875
interpolate by year time step 3
# 1.6997615098953247
interpolate by month time step 7
# 1.4450266361236572
interpolate by month time step 5
# 0.217756450176239


*** champion ***
interpolate by month time step 3
# 0.4332916736602783 25 units
# 0.15020249783992767 50 units 0.24396678805351257
# 0.08212704956531525 75 units
# 0.025785423815250397 75 units dense 15 , 1
# 0.09989678114652634 75 units dense 25 , 1
***  winner  # 0.016958503052592278 75 units dense 35 , 1 
*** end ***


binarize -1,0,1
year time step = 7
# 63.047935485839844
year time step = 5
# 59.16572189331055
year time step = 3
# 82.13848114013672
month time step = 7
# 11.621817588806152
month time step = 5
# 11.747268676757812
month time step = 3
# 15.268765449523926


binarize 0,1
year time step = 7
# 1.3483071327209473
year time step = 5
# 2.037656784057617
year time step = 3
# 4.179565906524658
month time step = 7
# 0.44504478573799133
month time step = 5
# 4.9152421951293945
month time step = 3
# 3.9173173904418945

binarize 1,2,3
year time step = 7
# 5.180117607116699
year time step = 5
# 4.566781520843506
year time step = 3
# 1.8392828702926636
month time step = 7
# 3.652085065841675
month time step = 5
# 4.432069778442383
month time step = 3
# 0.7650640606880188